In [1]:
from pathlib import Path

BASE = Path("/Users/joregan/Playing/rd_ctm_edit")

In [11]:
files = []
with open("/tmp/terror-files") as terrorf:
    for ff in terrorf.readlines():
        files.append(ff.strip())

In [50]:
def tidy(word, lc=True):
    if word[0:1] == '"':
        word = word[1:]
    if word[-1:] == '"':
        word = word[:-1]
    if word[-1:] in ".!?,;-":
        word = word[:-1]
    if lc:
        return word.lower()
    else:
        return word

In [20]:
import difflib

a = "afkanistan"
b = "Afghanistan."
def ratio(a, b):
    return difflib.SequenceMatcher(None, a, tidy(b)).ratio()

print(ratio(a, b))
print(ratio("male", "Mali"))
print(ratio("normader", "Nomader"))

0.8571428571428571
0.75
0.9333333333333333


In [22]:
def slurp(filename):
    lines = []
    with open(filename) as infile:
        for line in infile.readlines():
            lines.append(line.strip())
    return lines

In [23]:
testing = slurp("/Users/joregan/Playing/rd_ctm_edit/H001UFöU4")

In [24]:
testing[0:5]

['2442207150019781021 1 43.44 0.02 k 1.0 Jag sub',
 '2442207150019781021 1 49.04 0.42 härtalman 1.0 yrkar sub',
 '2442207150019781021 1 50.2 0.059 jag 1.0 bifall sub',
 '2442207150019781021 1 50.34 0.379 bifaller 1.0 till sub',
 '2442207150019781021 1 50.78 0.659 reservationen. 1.0 reservationen. cor']

In [25]:
CONJ = ["och", "men"]

In [151]:
def modify_pairs(sent_a, sent_b):
    def get_start_dur(sent_a, sent_b):
        start = sent_a[2]
        a_start = float(sent_a[2])
        a_dur = float(sent_a[3])
        b_start = float(sent_b[2])
        b_dur = float(sent_b[3])
        b_end = b_start + b_dur
        new_dur = b_end - a_start
        return start, "{:.3f}".format(new_dur)

    a = sent_a.split(" ")
    b = sent_b.split(" ")

    changed = False

    if a[4] == tidy(b[6]) or is_subst(a[4], tidy(b[6], False)):
        if a[6] == "<eps>":
            changed = True
            a[4] = a[6] = b[6]
            b[6] = "<eps>"
            a[7] = "cor"
            b[7] = "ins"
        if b[4] in CONJ:
            b[7] = "ins-conj"
    elif a[4] == "<eps>":
        if a[6] + b[6] == b[4]:
            changed = True
            joined = f"{a[6]}_{b[6]}"
            b[4] = b[6] = joined
            b[7] = "cor"
            a = []
    elif a[4] + b[4] == tidy(a[6]) and b[6] == "<eps>":
        print("a")
        changed = True
        start, end = get_start_dur(a, b)
        b[4] = b[6] = a[6]
        b[7] = "cor"
        b[2] = start
        b[3] = end
        a = []
    elif a[4] + b[4] == tidy(b[6]) and a[6] == "<eps>":
        print("b")
        changed = True
        start, end = get_start_dur(a, b)
        b[4] = b[6]
        b[7] = "cor"
        b[2] = start
        b[3] = end
        a = []
    if changed:
        return (" ".join(a), " ".join(b))
    else:
        return None

In [133]:
modify_pairs("2442207150019781021 1 58.4 0.78 invandringspolitiken 1.0 <eps> ins",
             "2442207150019781021 1 59.9 0.079 men 1.0 invandringspolitiken. sub")

('2442207150019781021 1 58.4 0.78 invandringspolitiken. 1.0 invandringspolitiken. cor',
 '2442207150019781021 1 59.9 0.079 men 1.0 <eps> ins-conj')

In [134]:
modify_pairs("2442207150019781021 1 97.12 0.0 <eps> 1.0 insats del",
             "2442207150019781021 1 97.18 0.739 insatsregeringen 1.0 regeringen sub")

('',
 '2442207150019781021 1 97.18 0.739 insats_regeringen 1.0 insats_regeringen cor')

In [135]:
modify_pairs("2442207150019781021 1 121.18 0.559 afkanistan 1.0 <eps> ins",
"2442207150019781021 1 122.62 0.079 och 1.0 Afghanistan. sub")

('2442207150019781021 1 121.18 0.559 Afghanistan. 1.0 Afghanistan. cor',
 '2442207150019781021 1 122.62 0.079 och 1.0 <eps> ins-conj')

In [152]:
modify_pairs("2442207150019781021 1 121.18 0.559 af 1.0 <eps> ins",
"2442207150019781021 1 122.62 0.079 ghanistan 1.0 Afghanistan. sub")

b


('', '2442207150019781021 1 121.18 1.519 Afghanistan. 1.0 Afghanistan. cor')

In [150]:
_SUBS = """
afkanistan Afghanistan
maly Mali
male Mali
övvik Ö-vik
abebégymnasiet ABB-gymnasiet
abeflygplan A/B-flygplan
abefs ABF:s
abfs ABF:s
abies ABS
abieffkurserna ABF-kursern
abief ABF
aczen Axén
adeärrkort ADR-kort
adeärr ADR
adiellbedömningen ADL-bedömningen
adiesselstationer ADSL-stationer
adihodepreparaten adhd-preparat
adihodemediciner adhd-mediciner
adihodeläkemedel adhd-läkemedel
adihodedroger adhd-droger
aduptioner adoptioner
aeff AF
aelless ALS
aenddeesstrategin ANDTS-strategin
aettaffisch A1-affisch
aendete ANDT
aendeteanvändandet ANDT-användandet
aendeteessfrågorna ANDTS-frågorna
aendeteesspolitiken ANDTS-politiken
aendeteessstrategin ANDTS-strategin
aendetefrågan ANDT-frågan
aendetefrågor ANDT-frågor
aendeteområdet ANDT-området
aendetessområdet ANDTS-området
aendetesspolitiskt ANDTS-politiskt
aendetesstrategi ANDTS-strategi
aendetestrategi ANDT-strategi
aendetestrategin ANDT-strategin
rutavdraget RUT-avdraget
sahäll Sahel
akim Aqim
alkaida_relaterade al-Qaida-relaterade
libien Libyen
malie Mali
kunskapoc kunskap_och
adisabeba Addis_Abeba
eus EU:s
gihadister jihadister
"""
def get_subs():
    subs = {}
    for l in _SUBS.split("\n"):
        if l.strip() == "":
            continue
        p = l.strip().split(" ")
        if not p[0] in subs:
            subs[p[0]] = []
        for part in p[1:]:
            subs[p[0]].append(part)
    return subs

In [148]:
def modify_single(sentence):
    parts = sentence.split(" ")
    if is_subst(parts[4], tidy(parts[6])):
        parts[4] = parts[6]
        parts[7] = "cor"
        return " ".join(parts)
    return sentence


In [62]:
SUBS = get_subs()

In [45]:
def is_subst(a, b):
    if a in SUBS:
        if b in SUBS[a]:
            return True
    return False

In [46]:
is_subst("male", "Mali")

True

In [64]:
%pip install more-itertools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [65]:
import more_itertools

In [68]:
for pair in more_itertools.windowed(testing[0:6], 2):
    print(pair)

('2442207150019781021 1 43.44 0.02 k 1.0 Jag sub', '2442207150019781021 1 49.04 0.42 härtalman 1.0 yrkar sub')
('2442207150019781021 1 49.04 0.42 härtalman 1.0 yrkar sub', '2442207150019781021 1 50.2 0.059 jag 1.0 bifall sub')
('2442207150019781021 1 50.2 0.059 jag 1.0 bifall sub', '2442207150019781021 1 50.34 0.379 bifaller 1.0 till sub')
('2442207150019781021 1 50.34 0.379 bifaller 1.0 till sub', '2442207150019781021 1 50.78 0.659 reservationen. 1.0 reservationen. cor')
('2442207150019781021 1 50.78 0.659 reservationen. 1.0 reservationen. cor', '2442207150019781021 1 52.76 0.06 På 1.0 På cor')


In [149]:
def partition(lines):
    def has_final(line):
        FINAL = ".!?"
        parts = line.strip().split(" ")
        piece = parts[6]
        if piece.endswith('"'):
            piece = piece[:-1]
        return piece[-1:] in FINAL
    
    def is_capital(line):
        parts = line.strip().split(" ")
        piece = parts[6]
        return piece[0:1].isupper()
    
    def splittable(a, b):
        return has_final(a) and is_capital(b)

    sentences = []
    current = []
    last_mod = ""

    for pair in more_itertools.windowed(lines + ["END"], 2):
        first = modify_single(pair[0])
        second = modify_single(pair[1])
        if last_mod != "":
            second = last_mod
        last_mod = ""
        
        if second == "END":
            current.append(first)
            sentences.append(current[:])
        else:
            mod = modify_pairs(first, second)
            if mod is not None:
                first = mod[0]
                second = mod[1]
                last_mod = second
            elif splittable(first, second):
                current.append(first)
                sentences.append(current[:])
                current = []
            else:
                current.append(first)
    return sentences

In [90]:
parting = partition(testing)

In [93]:
ttt = parting[0]
ttt

['2442207150019781021 1 43.44 0.02 k 1.0 Jag sub',
 '2442207150019781021 1 49.04 0.42 härtalman 1.0 yrkar sub',
 '2442207150019781021 1 50.2 0.059 jag 1.0 bifall sub',
 '2442207150019781021 1 50.34 0.379 bifaller 1.0 till sub',
 '2442207150019781021 1 50.78 0.659 reservationen. 1.0 reservationen. cor']

In [102]:
def sentence_correctness(sentence):
    length = len(sentence)
    score = 0
    for word in sentence:
        parts = word.split(" ")
        if parts[-1].startswith("cor"):
            score += 1
    return score / length

In [107]:
sentence_correctness(parting[4])

1.0

In [123]:
parting[25]

['2442207150019781021 1 207.8 0.339 Mali 1.0 Mali cor',
 '2442207150019781021 1 208.54 0.08 har 1.0 har cor',
 '2442207150019781021 1 208.7 0.38 drabbats 1.0 drabbats cor',
 '2442207150019781021 1 209.079 0.0 <eps> 1.0 hårt. del']

In [115]:
parting[26]

['2442207150019781021 1 209.079 0.0 <eps> 1.0 Mali del',
 '2442207150019781021 1 209.12 0.259 hårt 1.0 är sub',
 '2442207150019781021 1 209.48 0.219 mali 1.0 också sub',
 '2442207150019781021 1 209.8 0.099 ett 1.0 ett cor',
 '2442207150019781021 1 209.96 0.199 land 1.0 land cor',
 '2442207150019781021 1 210.22 0.08 som 1.0 som cor',
 '2442207150019781021 1 210.38 0.14 också 1.0 <eps> ins',
 '2442207150019781021 1 210.6 0.64 utrikesutskottet 1.0 utrikesutskottet cor',
 '2442207150019781021 1 211.239 0.0 <eps> 1.0 har del',
 '2442207150019781021 1 211.32 0.12 haft 1.0 haft cor',
 '2442207150019781021 1 211.48 0.36 möjlighet 1.0 möjlighet cor',
 '2442207150019781021 1 211.92 0.1 att 1.0 att cor',
 '2442207150019781021 1 212.08 0.519 besöka 1.0 besöka cor',
 '2442207150019781021 1 212.92 0.16 under 1.0 under cor',
 '2442207150019781021 1 213.14 0.08 en 1.0 en cor',
 '2442207150019781021 1 213.26 0.06 av 1.0 av cor',
 '2442207150019781021 1 213.4 0.239 sina 1.0 sina cor',
 '2442207150019781

In [117]:
parting[28]

['2442207150019781021 1 227.62 0.039 Vi 1.0 Vi cor',
 '2442207150019781021 1 227.8 0.639 socialdemokrater 1.0 socialdemokrater cor',
 '2442207150019781021 1 228.5 0.08 ser 1.0 ser cor',
 '2442207150019781021 1 228.62 0.06 det 1.0 det cor',
 '2442207150019781021 1 228.72 0.06 som 1.0 som cor',
 '2442207150019781021 1 228.82 0.06 en 1.0 en cor',
 '2442207150019781021 1 229.0 0.819 förutsättning 1.0 förutsättning cor',
 '2442207150019781021 1 229.86 0.099 att 1.0 att cor',
 '2442207150019781021 1 229.98 0.06 det 1.0 det cor',
 '2442207150019781021 1 230.08 0.159 finns 1.0 finns cor',
 '2442207150019781021 1 230.28 0.06 en 1.0 en cor',
 '2442207150019781021 1 230.44 0.36 bred 1.0 bred cor',
 '2442207150019781021 1 230.88 0.48 majoritet 1.0 majoritet cor',
 '2442207150019781021 1 231.42 0.02 i 1.0 när sub',
 '2442207150019781021 1 231.52 0.319 kammaren 1.0 kammaren cor',
 '2442207150019781021 1 231.88 0.08 för 1.0 <eps> ins',
 '2442207150019781021 1 231.98 0.12 att 1.0 <eps> ins',
 '2442207

In [118]:
def find_terror_segs(segments):
    terror = []
    def ismatch(word):
        return "terror" in word or "teror" in word

    for segment in segments:
        for line in segment:
            parts = line.split(" ")
            if ismatch(parts[4]) or ismatch(parts[6].lower()):
                terror.append(segment)
    return terror
            

In [120]:
tsegs = find_terror_segs(parting)

In [121]:
len(tsegs)

9

In [124]:
tsegs

[['2442207150019781021 1 305.86 0.439 malisk 1.0 Malis sub',
  '2442207150019781021 1 306.48 0.74 suveränitet, 1.0 suveränitet, cor',
  '2442207150019781021 1 307.36 0.36 enhet 1.0 enhet cor',
  '2442207150019781021 1 307.76 0.079 och 1.0 och cor',
  '2442207150019781021 1 307.9 0.66 terrortuella 1.0 territoriella sub',
  '2442207150019781021 1 308.72 0.5 integritet 1.0 integritet cor',
  '2442207150019781021 1 309.6 0.259 var 1.0 var cor',
  '2442207150019781021 1 310.02 0.4 uppenbart 1.0 uppenbart cor',
  '2442207150019781021 1 310.48 0.339 hotad 1.0 hotad cor',
  '2442207150019781021 1 311.16 0.159 efter 1.0 efter cor',
  '2442207150019781021 1 311.34 0.1 att 1.0 att cor',
  '2442207150019781021 1 311.48 0.56 islamisterna 1.0 islamister sub',
  '2442207150019781021 1 312.1 0.319 intagit 1.0 intagit cor',
  '2442207150019781021 1 312.48 0.299 stora 1.0 stora cor',
  '2442207150019781021 1 312.88 0.259 delar 1.0 delar cor',
  '2442207150019781021 1 313.22 0.059 av 1.0 av cor',
  '2442

In [127]:
def get_readable_sentence(sentence):
    outwords = []
    for word in sentence:
        parts = word.split(" ")
        if parts[-1].startswith("cor"):
            outwords.append(parts[6])
        else:
            if parts[4] == "<eps>":
                outwords.append(f"+({parts[6]})")
            elif parts[6] == "<eps>":
                outwords.append(f"-({parts[4]})")
            else:
                outwords.append(f"({parts[4]}/{parts[6]})")
    return " ".join(outwords)

In [128]:
get_readable_sentence(tsegs[0])

'(malisk/Malis) suveränitet, enhet och (terrortuella/territoriella) integritet var uppenbart hotad efter att (islamisterna/islamister) intagit stora delar av (maliskt/Malis) territorium.'

In [129]:
get_readable_sentence(parting[28])

'Vi socialdemokrater ser det som en förutsättning att det finns en bred majoritet (i/när) kammaren -(för) -(att) -(ta) (den/tar) (här/denna) (arten/art) av +(beslut,) (beslut/och) (de/det) finns +(det) också.'

In [ ]:
OUTPATH = Path("/tmp/outfiles")
for file in files:
    OUTFILE = OUTPATH / file.stem
    TERRORFILE = OUTPATH / f"TERROR_{file.stem}"
    with open(OUTFILE, "w") as outputfile, open(TERRORFILE, "w") as terrorfile:
        lines = slurp(file)
        partitions = partition(lines)
        for ptn in partitions:
            for line in ptn:
                outputfile.write(line + "\n")
            outputfile.write("\n")
        terror_segs = find_terror_segs(partitions)
        for tsg in terror_segs:
            for line in tsg:
                terrorfile.write(line + "\n")
            terrorfile.write("\n")